In [2]:
#⁠Populating Store_dimension table based on country, state, city and store OLTP tables

import pandas as pd
from sqlalchemy import create_engine,Table,Column,Integer,String,MetaData,ForeignKey
engine = create_engine('sqlite:///store_dim.db')
meta = MetaData()
con = engine.connect()
meta.create_all(engine)

In [3]:
country = Table('country',meta,
               Column('country_id',Integer,primary_key=True),
               Column('country_name',String)
               )

In [4]:
state = Table('state',meta,
               Column('state_id',Integer,primary_key=True),
               Column('state_name',String),
              Column('country_id',Integer,ForeignKey('country.country_id'))
               )

In [5]:
city = Table('city',meta,
               Column('city_id',Integer,primary_key=True),
               Column('city_name',String),
              Column('state_id',Integer,ForeignKey('state.state_id'))
               )

In [26]:
store = Table('store',meta,
             Column('Store_id',Integer,primary_key = True),
             Column('Store_name',String),
             Column('city_id',Integer))

In [9]:
store_dim = Table('store_dim',meta,
                 Column('Store_id',Integer,primary_key=True),
                  Column('Store_name',String),
                  Column('city_name',String,ForeignKey('city.city_name')),
                  Column('State_name',String,ForeignKey('state.state_name')),
                  Column('country_name',String,ForeignKey('country.country_name'))
                 )

In [27]:
meta.create_all(engine)

In [30]:
store_df = pd.read_sql_table('store',engine)
city_df = pd.read_sql_table('city',engine)
country_df = pd.read_sql_table('country',engine)
state_df = pd.read_sql_table('state',engine)

In [32]:
merged_df = pd.merge(store_df,city_df,on = 'city_id', how = 'inner')
merged_df = pd.merge(merged_df,state_df,on = 'state_id',how = 'inner')
merged_df = pd.merge(merged_df,country_df,on = 'country_id')

In [35]:
merged_df.columns

Index(['Store_id', 'Store_name', 'city_id', 'city_name', 'state_id',
       'state_name', 'country_id', 'country_name'],
      dtype='object')

In [37]:
merged_df[['Store_id','Store_name','city_name','state_name','country_name']].to_sql('store_dim',engine,index=False,if_exists='replace')

34